In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from fbprophet import Prophet
import pandas as pd
%matplotlib inline 

dataset11_2 = pd.read_csv("../input/sep-dataset-fat97/20161231_20191128_vfat.csv",infer_datetime_format=True,
                       parse_dates = ['DATA'])
dataset12_1 = pd.read_csv("../input/sep-dataset-fat98/20161231_20191215_vfat.csv",infer_datetime_format=True,
                       parse_dates = ['DATA'])
dataset01_2 = pd.read_csv("../input/sep-dataset-fat01/20161231_20200102_vfat.csv",infer_datetime_format=True,
                       parse_dates = ['DATA'])
dataset01_8 = pd.read_csv("../input/sep-dataset-fat08/20161231_20200108_vfat.csv",infer_datetime_format=True,
                       parse_dates = ['DATA'])

dataset = dataset01_8.copy()

daily_df = dataset[['DATA','TRN_TOTAL']][dataset['FL_ATIVO'] == 1]
#daily_df = daily_df[daily_df['DATA'] <= '2019-04-08']
#daily_df = daily_df[daily_df['DATA'] >= '2016-12-31']
daily_df = daily_df.groupby('DATA')['TRN_TOTAL'].sum()
daily_df = daily_df.resample('D').sum()
df = daily_df.reset_index()
df.columns = ['ds', 'y']
df.head()
df.tail()

hldys1 = pd.DataFrame({
  'holiday': 'queda',
  'ds': pd.to_datetime([
                        '2017-02-27', '2017-02-28', '2017-04-14', '2017-04-16', '2017-04-21', '2017-05-01', '2017-06-15', 
                        '2017-09-07', '2017-10-12', '2017-11-02', '2017-11-15',
                        '2018-02-12', '2018-02-13', '2018-03-30', '2018-04-01', '2018-05-01', '2018-05-31', '2018-09-07',
                        '2018-10-12', '2018-11-02', '2018-11-15', 
                        '2019-03-04', '2019-03-05', '2019-04-19', '2019-04-21', '2019-05-01', '2019-06-20', '2019-09-07',
                        '2019-10-12', '2019-11-02', '2019-11-15',
                        '2020-02-24', '2020-02-25', '2020-04-10', '2020-04-12', '2020-04-21', '2020-05-01', '2020-06-11', 
                        '2020-09-07', '2020-10-12', '2020-11-02', '2020-11-15']),
  'lower_window': 0,
  'upper_window': 0,
})

hldys2 = pd.DataFrame({
  'holiday': 'alta_sep',
  'ds': pd.to_datetime(['2017-12-20', '2018-12-20', '2019-12-20', '2020-12-18']),
  'lower_window': 0,
  'upper_window': 2,
})

hldys3 = pd.DataFrame({
  'holiday': 'especial',
  'ds': pd.to_datetime(['2017-12-25', '2018-01-01', '2018-12-25', '2019-01-01', '2019-12-25', '2020-01-01','2020-12-25', '2021-01-01']),
  'lower_window': 0,
  'upper_window': 0,
})

hldys4 = pd.DataFrame({
  'holiday': 'alta',
  'ds': pd.to_datetime(['2017-02-24', '2017-03-01', '2017-03-02', '2017-06-10', '2017-08-12', '2017-11-24', #'2017-05-12',
                        '2018-02-09', '2018-02-14', '2018-02-15', '2018-06-12', '2018-08-11', '2018-11-23',#'2018-05-11',
                        '2019-03-01', '2019-03-06', '2019-03-07', '2019-05-11', '2019-06-12', '2019-08-09','2019-11-29',
                        '2020-02-21', '2020-03-26', '2020-03-27', '2020-05-08', '2020-06-12', '2020-08-07','2020-11-27']),
  'lower_window': 0,
  'upper_window': 0,
})
#'2017-02-24','2017-03-01','2017-03-02',
#'2018-02-09','2018-02-14','2018-02-15',
#'2019-03-01','2019-03-06','2019-03-07',
hldys = pd.concat((hldys1, hldys2, hldys3, hldys4))


prediction_size = 488
train_df = df.copy()
#train_df = df[:-prediction_size]
train_df['y'] = np.log(train_df['y'])
train_df.tail(n=6)

In [ ]:
j = 31 #16  # weekly seasonality prior_scale
i = 46 #31  # monthly seasonality prior_scale
b = 31 #16  # seasonality prior_scale
c = 46 #50  # holidays seasonality prior_scale
m = Prophet(yearly_seasonality=False,weekly_seasonality=False,growth = 'linear',
            daily_seasonality=False, holidays = hldys, holidays_prior_scale = c,
           seasonality_prior_scale = b, seasonality_mode = 'additive',
           changepoint_prior_scale = 0.99 , changepoint_range = 0.59
           ).add_seasonality(
            name='w',period=7,fourier_order=10, prior_scale = j).add_seasonality(
            name='m',period=30.5,fourier_order=20, prior_scale = i).add_seasonality(
            name='ano',period=365,fourier_order=40, prior_scale = 1.3*i)
m.fit(train_df)

future = m.make_future_dataframe(periods=prediction_size)
future.tail(n=3)

forecast = m.predict(future)
a = forecast.set_index('ds')[['yhat']].join(df.set_index('ds'))
a['yhat'] = np.exp(a['yhat'])

print('done')

In [ ]:
b = dataset.set_index('DATA')['2020-01-01':'2020-01-08']['TRN_TOTAL'].sum()
c = a['2020-01-09':'2020-01-31']['yhat'].sum()
est = b + c
print(b)
print(round(est))
#print(a['2020-01-01':'2020-01-31']['yhat'])

In [ ]:
import datetime
import matplotlib.pyplot
teste = a.copy()
teste = teste.resample('M').sum()
teste = teste.reset_index()
k = teste['ds'].astype(str)
fig = matplotlib.pyplot.gcf()
fig.set_size_inches(15.5, 4.5)
plt.plot(k,teste[['yhat','y']])
plt.xticks(rotation='vertical')
plt.show()
print(teste[['yhat','y']].astype(int))
#teste.to_excel('projecao_trn_2020.xlsx',index=False)

In [ ]:
print('jun18',round(dataset.set_index('DATA')['2018-06-01':'2018-06-30']['TRN_TOTAL'].sum()))
print('jul18',round(dataset.set_index('DATA')['2018-07-01':'2018-07-31']['TRN_TOTAL'].sum()))
print('ago18',round(dataset.set_index('DATA')['2018-08-01':'2018-08-31']['TRN_TOTAL'].sum()))
print('set18',round(dataset.set_index('DATA')['2018-09-01':'2018-09-30']['TRN_TOTAL'].sum()))
print('out18',round(dataset.set_index('DATA')['2018-10-01':'2018-10-31']['TRN_TOTAL'].sum()))
print('nov18',round(dataset.set_index('DATA')['2018-11-01':'2018-11-30']['TRN_TOTAL'].sum()))
print('dez18',round(dataset.set_index('DATA')['2018-12-01':'2018-12-31']['TRN_TOTAL'].sum()))
print('')
print('jan19',round(dataset.set_index('DATA')['2019-01-01':'2019-01-31']['TRN_TOTAL'].sum()))
print('fev19',round(dataset.set_index('DATA')['2019-02-01':'2019-02-28']['TRN_TOTAL'].sum()))
print('mar19',round(dataset.set_index('DATA')['2019-03-01':'2019-03-31']['TRN_TOTAL'].sum()))
print('abr19',round(dataset.set_index('DATA')['2019-04-01':'2019-04-30']['TRN_TOTAL'].sum()))
print('mai19',round(dataset.set_index('DATA')['2019-05-01':'2019-05-31']['TRN_TOTAL'].sum()))
print('jun19',round(dataset.set_index('DATA')['2019-06-01':'2019-06-30']['TRN_TOTAL'].sum()))
print('jul19',round(dataset.set_index('DATA')['2019-07-01':'2019-07-31']['TRN_TOTAL'].sum()))
print('ago19',round(dataset.set_index('DATA')['2019-08-01':'2019-08-31']['TRN_TOTAL'].sum()))
print('set19',round(dataset.set_index('DATA')['2019-09-01':'2019-09-30']['TRN_TOTAL'].sum()))
print('out19',round(dataset.set_index('DATA')['2019-10-01':'2019-10-31']['TRN_TOTAL'].sum()))
print('nov19',round(dataset.set_index('DATA')['2019-11-01':'2019-11-30']['TRN_TOTAL'].sum()))
print('dez19',round(dataset.set_index('DATA')['2019-12-01':'2019-12-31']['TRN_TOTAL'].sum()))
print('')
print('jan20',round(a['2020-01-01':'2020-01-31']['yhat'].sum()))
print('fev20',round(a['2020-02-01':'2020-02-29']['yhat'].sum()))
print('mar20',round(a['2020-03-01':'2020-03-31']['yhat'].sum()))
print('abr20',round(a['2020-04-01':'2020-04-30']['yhat'].sum()))
print('mai20',round(a['2020-05-01':'2020-05-31']['yhat'].sum()))
print('jun20',round(a['2020-06-01':'2020-06-30']['yhat'].sum()))
print('jul20',round(a['2020-07-01':'2020-07-31']['yhat'].sum()))
print('ago20',round(a['2020-08-01':'2020-08-31']['yhat'].sum()))
print('set20',round(a['2020-09-01':'2020-09-30']['yhat'].sum()))
print('out20',round(a['2020-10-01':'2020-10-31']['yhat'].sum()))
print('nov20',round(a['2020-11-01':'2020-11-30']['yhat'].sum()))
print('dez20',round(a['2020-12-01':'2020-12-31']['yhat'].sum()))

In [ ]:
pd.plotting.register_matplotlib_converters()
m.plot_components(forecast)

In [ ]:
from fbprophet.plot import add_changepoints_to_plot
pd.plotting.register_matplotlib_converters()
fig = m.plot(forecast)
h = add_changepoints_to_plot(fig.gca(), m, forecast)